## Dog breed classification via ResNet-50 transfer learning version 2

Tutorial for dog breed classification via ResNet-50 transfer learning in pytorch by using the dataset stored in the home directory

### Step1: Download data

In [ ]:
# change directory
notebook_id = %env TRAINING_NOTEBOOK_ID
%cd '/mnt/notebooks/{notebook_id}/Platform_handson/dog_breed_classification'

In [ ]:
# save data in /home/data
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar -P /home/data

In [ ]:
# extract tar file
n_files = !tar tf /home/data/images.tar | grep .jpg | wc -l
!tar xvf /home/data/images.tar -C /home/data/ | pv -l -s {n_files[0]} > /dev/null

### Step 2: Import library

In [ ]:
import os
import io
import numpy as np
from glob import glob             
from PIL import Image, ImageFile

import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models

import matplotlib.pyplot as plt                        
%matplotlib inline

ImageFile.LOAD_TRUNCATED_IMAGES = True

# check if CUDA is available
use_cuda = torch.cuda.is_available()
print(use_cuda)

### Step 3: Vasualize data

In [ ]:
# load filenames for dog images
dog_files = list(glob("/home/data/Images/*/*"))

# print number of images in dataset
print('There are %d total dog images.' % len(dog_files))

In [ ]:
def image_to_tensor(img):
    '''
    As per Pytorch documentations: All pre-trained models expect input images normalized in the same way, 
    i.e. mini-batches of 3-channel RGB images
    of shape (3 x H x W), where H and W are expected to be at least 224. 
    The images have to be loaded in to a range of [0, 1] and 
    then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]. 
    '''
    img = img.convert('RGB')
    transformations = transforms.Compose([transforms.Resize(size=224),
                                          transforms.CenterCrop((224,224)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])
    image_tensor = transformations(img)[:3,:,:].unsqueeze(0)
    return image_tensor


# helper function for un-normalizing an image
# and converting it from a Tensor image to a NumPy image for display
def im_convert(tensor):
    """ Display a tensor as an image. """
    
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image

In [ ]:
# show one of images
dog_image = Image.open('/home/data/Images/n02085620-Chihuahua/n02085620_431.jpg')
plt.imshow(dog_image)
plt.show()

In [ ]:
# show one of images after conversion
test_tensor = image_to_tensor(dog_image)
print(test_tensor.shape)
plt.imshow(im_convert(test_tensor))

### Step 4: Preprocess data

In [ ]:
# define training and validation data directories
data_dir = '/home/data/Images/'

# how many samples per batch to load
batch_size = 16

# number of subprocesses to use for data loading
num_workers = 2

# difine size of validation data
valid_size = 0.2

In [ ]:
# convert data to a normalized torch.FloatTensor
train_transform = transforms.Compose([transforms.Resize(size=224),
                                transforms.CenterCrop((224,224)),
                                transforms.RandomHorizontalFlip(), # randomly flip and rotate
                                transforms.RandomRotation(10),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

valid_transform = transforms.Compose([transforms.Resize(size=224),
                                transforms.CenterCrop((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
def load_split_train_test(data_dir, valid_size, batch_size, num_workers):
    """ Split dataset into train and validation set """
    
    train_data = datasets.ImageFolder(data_dir,       
                    transform=train_transform)
    valid_data = datasets.ImageFolder(data_dir,
                    transform=valid_transform)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    
    from torch.utils.data.sampler import SubsetRandomSampler
    
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size, num_workers=num_workers)
    validloader = torch.utils.data.DataLoader(valid_data,
                   sampler=valid_sampler, batch_size=batch_size, num_workers=num_workers)
    
    return trainloader, validloader

trainloader, validloader = load_split_train_test(data_dir, valid_size, batch_size, num_workers)

In [ ]:
class_names = trainloader.dataset.classes
nb_classes = len(class_names)

print("Number of classes:", nb_classes)
print("\nClass names: \n\n", class_names)

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(trainloader))

for image, label in zip(inputs, classes): 
    image = image.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)
     
    fig = plt.figure(figsize=(12,3))
    plt.imshow(image)
    plt.title(class_names[label])

### Step 5: Training

In [ ]:
# specify model architecture (ResNet-50)
model = models.resnet50(pretrained=True)

In [ ]:
# freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# replace the last fully connected layer with a Linnear layer with 120 out features
model.fc = nn.Linear(2048, 120)

if use_cuda:
    model = model.cuda()

In [ ]:
# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
def train(n_epochs, trainloader, validloader, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = 3.877533 #np.Inf
    
    if os.path.exists(save_path):
        model.load_state_dict(torch.load(save_path))
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training, validation loss and validation accuracy
        train_loss = 0.0
        valid_loss = 0.0
        total = 0
        correct = 0
        
        # train the model
        model.train()
        for data, target in trainloader:
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)
            
        # validate the model
        model.eval()
        for data, target in validloader:
            if use_cuda:
                data, target = data.cuda(), target.cuda()
    
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            # count number of correct labels
            _, preds_tensor = torch.max(output, 1)
            total += target.size(0)
            correct += (preds_tensor == target).sum().item()
            
        # calculate average losses
        train_loss = train_loss/len(trainloader.dataset)
        valid_loss = valid_loss/len(validloader.dataset)
        # calculate accuracy
        valid_accuracy = correct/total
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation accuracy: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            valid_accuracy
            ))
        
        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

In [ ]:
# train the model
n_epochs = 5
train(n_epochs, trainloader, validloader, model, optimizer, criterion, use_cuda, './model/dog_model.pt')

### Step 6: Inference

In [ ]:
from PIL import Image
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models

# load the model for CPU
device = torch.device('cpu')
dog_model = models.resnet50(pretrained=True)

# freeze parameters so we don't backprop through them
for param in dog_model.parameters():
    param.requires_grad = False
    
# replace the last fully connected layer with a Linnear layer with 120 out features
dog_model.fc = nn.Linear(2048, 120)

dog_model.load_state_dict(torch.load('./model/dog_model.pt', map_location=device))

In [ ]:
def image_to_tensor(img):
    '''
    As per Pytorch documentations: All pre-trained models expect input images normalized in the same way, 
    i.e. mini-batches of 3-channel RGB images
    of shape (3 x H x W), where H and W are expected to be at least 224. 
    The images have to be loaded in to a range of [0, 1] and 
    then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]. 
    '''
    img = img.convert('RGB')
    transformations = transforms.Compose([transforms.Resize(size=224),
                                          transforms.CenterCrop((224,224)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])
    image_tensor = transformations(img)[:3,:,:].unsqueeze(0)
    return image_tensor

In [ ]:
# list of class names
class_names = ['CHIHUAHUA',
 'JAPANESE SPANIEL',
 'MALTESE DOG',
 'PEKINESE',
 'SHIH-TZU',
 'BLENHEIM SPANIEL',
 'PAPILLON',
 'TOY TERRIER',
 'RHODESIAN RIDGEBACK',
 'AFGHAN HOUND',
 'BASSET',
 'BEAGLE',
 'BLOODHOUND',
 'BLUETICK',
 'BLACK-AND-TAN COONHOUND',
 'WALKER HOUND',
 'ENGLISH FOXHOUND',
 'REDBONE',
 'BORZOI',
 'IRISH WOLFHOUND',
 'ITALIAN GREYHOUND',
 'WHIPPET',
 'IBIZAN HOUND',
 'NORWEGIAN ELKHOUND',
 'OTTERHOUND',
 'SALUKI',
 'SCOTTISH DEERHOUND',
 'WEIMARANER',
 'STAFFORDSHIRE BULLTERRIER',
 'AMERICAN STAFFORDSHIRE TERRIER',
 'BEDLINGTON TERRIER',
 'BORDER TERRIER',
 'KERRY BLUE TERRIER',
 'IRISH TERRIER',
 'NORFOLK TERRIER',
 'NORWICH TERRIER',
 'YORKSHIRE TERRIER',
 'WIRE-HAIRED FOX TERRIER',
 'LAKELAND TERRIER',
 'SEALYHAM TERRIER',
 'AIREDALE',
 'CAIRN',
 'AUSTRALIAN TERRIER',
 'DANDIE DINMONT',
 'BOSTON BULL',
 'MINIATURE SCHNAUZER',
 'GIANT SCHNAUZER',
 'STANDARD SCHNAUZER',
 'SCOTCH TERRIER',
 'TIBETAN TERRIER',
 'SILKY TERRIER',
 'SOFT-COATED WHEATEN TERRIER',
 'WEST HIGHLAND WHITE TERRIER',
 'LHASA',
 'FLAT-COATED RETRIEVER',
 'CURLY-COATED RETRIEVER',
 'GOLDEN RETRIEVER',
 'LABRADOR RETRIEVER',
 'CHESAPEAKE BAY RETRIEVER',
 'GERMAN SHORT-HAIRED POINTER',
 'VIZSLA',
 'ENGLISH SETTER',
 'IRISH SETTER',
 'GORDON SETTER',
 'BRITTANY SPANIEL',
 'CLUMBER',
 'ENGLISH SPRINGER',
 'WELSH SPRINGER SPANIEL',
 'COCKER SPANIEL',
 'SUSSEX SPANIEL',
 'IRISH WATER SPANIEL',
 'KUVASZ',
 'SCHIPPERKE',
 'GROENENDAEL',
 'MALINOIS',
 'BRIARD',
 'KELPIE',
 'KOMONDOR',
 'OLD ENGLISH SHEEPDOG',
 'SHETLAND SHEEPDOG',
 'COLLIE',
 'BORDER COLLIE',
 'BOUVIER DES FLANDRES',
 'ROTTWEILER',
 'GERMAN SHEPHERD',
 'DOBERMAN',
 'MINIATURE PINSCHER',
 'GREATER SWISS MOUNTAIN DOG',
 'BERNESE MOUNTAIN DOG',
 'APPENZELLER',
 'ENTLEBUCHER',
 'BOXER',
 'BULL MASTIFF',
 'TIBETAN MASTIFF',
 'FRENCH BULLDOG',
 'GREAT DANE',
 'SAINT BERNARD',
 'ESKIMO DOG',
 'MALAMUTE',
 'SIBERIAN HUSKY',
 'AFFENPINSCHER',
 'BASENJI',
 'PUG',
 'LEONBERG',
 'NEWFOUNDLAND',
 'GREAT PYRENEES',
 'SAMOYED',
 'POMERANIAN',
 'CHOW',
 'KEESHOND',
 'BRABANCON GRIFFON',
 'PEMBROKE',
 'CARDIGAN',
 'TOY POODLE',
 'MINIATURE POODLE',
 'STANDARD POODLE',
 'MEXICAN HAIRLESS',
 'DINGO',
 'DHOLE',
 'AFRICAN HUNTING DOG']

In [ ]:
def predict_breed(img, use_cuda=False):
    """ load the image and return the predicted breed """
    image_tensor = image_to_tensor(img)
    
    # get outputs
    if use_cuda:
        image_tensor = image_tensor.cuda()
    
    dog_model.eval()
    output = dog_model(image_tensor)
    
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    pred = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())
    result = class_names[pred]
    
    return result

def display_image(img, title="Title"):
    plt.title(title)
    plt.imshow(img)
    plt.show()
    
def dog_classifier(img):
    predicted_breed = predict_breed(img)
    display_image(img, title=f"Predicted:{predicted_breed}")
        
    print("Your breed is most likley ...")
    print(predicted_breed)

In [ ]:
import matplotlib.pyplot as plt                        
%matplotlib inline

img = Image.open('./sample/test_french_bull.jpg')
dog_classifier(img)

In [ ]:
img = Image.open('./sample/test_bull_mastif.jpg')
dog_classifier(img)